<a href="https://colab.research.google.com/github/FoxCoder-hub/GA-TSP/blob/main/TSP_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library installation

In [ ]:
!pip install openpyxl

import pandas as pd
import numpy as np
import random
import time
from google.colab import files

# Parameters def

In [ ]:
POP_SIZE = 200
GENERATIONS = 500
TOURNAMENT_K = 5
MUTATION_RATE = 0.15
ELITISM = 10
RANDOM_SEED = 42

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# Read the uploaded file xlsx or csv and convert it to distance matrix

In [ ]:
# 1. First upload your file
uploaded = files.upload()

# 2. Get the filename
filename = list(uploaded.keys())[0]
print(f"Uploaded: {filename}")

# 3. Process your file (optional)
if filename.endswith('.csv'):
    df = pd.read_csv(filename)
elif filename.endswith(('.xlsx', '.xls')):
    df = pd.read_excel(filename)

Saving Distance_Matrix_USA (1).xlsx to Distance_Matrix_USA (1).xlsx
Uploaded: Distance_Matrix_USA (1).xlsx


In [ ]:
def read_distance_matrix(uploaded_file):
    # Display the dataframe info
    print("DataFrame shape:", df.shape)
    print("\nFirst few rows of the dataframe:")
    print(df.head())

    # Extract city names (assuming first column contains city names)
    cities = df.iloc[:, 0].tolist()

    # Extract distance matrix (assuming the rest of the dataframe is the distance matrix)
    # Remove the first column (city names) and convert to numpy array
    dist_matrix = df.iloc[:, 1:].values

    print(f"\nNumber of cities: {len(cities)}")
    print(f"Distance matrix shape: {dist_matrix.shape}")
    print(f"Cities: {cities}")

    return cities, dist_matrix

# 3. Use the function to read the distance matrix
cities, dist_matrix = read_distance_matrix(uploaded)

DataFrame shape: (27, 28)

First few rows of the dataframe:
        Unnamed: 0  Albuquerque, NM  Atlanta, GA  Austin, TX  Baltimore, MD  \
0  Albuquerque, NM                0         1494        1330           1295   
1      Atlanta, GA             1494            0        2947           1175   
2       Austin, TX             1330         2947           0           2155   
3    Baltimore, MD             1295         1175        2155              0   
4       Boston, MA             1838         2006        1785            579   

   Boston, MA  Charlotte, NC  Chicago, IL  Cleveland, OH  Columbus, OH  ...  \
0        1838           2369          666           1438           530  ...   
1        2006            389         2934            762          2099  ...   
2        1785           1221         2813           1329          1700  ...   
3         579            692         1380           2262           264  ...   
4           0           1951         1004           2346          2931

# GA Functions

In [ ]:
def route_distance(route, dist_matrix):
    d = sum(dist_matrix[route[i], route[i+1]] for i in range(len(route)-1))
    d += dist_matrix[route[-1], route[0]]
    return float(d)

def initial_population(n_cities, pop_size, start_idx):
    population = []
    nodes = [i for i in range(n_cities) if i != start_idx]
    for _ in range(pop_size):
        perm = nodes.copy()
        random.shuffle(perm)
        population.append([start_idx] + perm)
    return population

def tournament_selection(pop, fitnesses, k):
    selected = random.sample(range(len(pop)), k)
    best = min(selected, key=lambda i: fitnesses[i])
    return pop[best].copy()

def ordered_crossover(p1, p2, start_idx):
    n = len(p1)
    child = [-1] * n
    child[0] = start_idx
    a, b = random.randint(1, n-2), random.randint(1, n-1)
    if b <= a:
        a, b = b, a+1
    child[a:b] = p1[a:b]

    p2_iter = [x for x in p2[1:] if x not in child]
    pos = 1
    for v in p2_iter:
        while pos < n and child[pos] != -1:
            pos += 1
        if pos < n:
            child[pos] = v
            pos += 1
    return child

def mutate_swap(route, mutation_rate):
    n = len(route)
    for i in range(1, n):
        if random.random() < mutation_rate:
            j = random.randint(1, n-1)
            route[i], route[j] = route[j], route[i]

def genetic_tsp(dist_matrix, start_idx, pop_size=POP_SIZE, generations=GENERATIONS):
    n = dist_matrix.shape[0]
    pop = initial_population(n, pop_size, start_idx)
    best_route, best_dist = None, float('inf')

    for gen in range(generations):
        fitnesses = [route_distance(r, dist_matrix) for r in pop]
        min_idx = int(np.argmin(fitnesses))
        if fitnesses[min_idx] < best_dist:
            best_dist = fitnesses[min_idx]
            best_route = pop[min_idx].copy()

        new_pop = []
        sorted_idx = np.argsort(fitnesses)
        for e in range(min(ELITISM, len(sorted_idx))):
            new_pop.append(pop[int(sorted_idx[e])].copy())

        while len(new_pop) < pop_size:
            p1 = tournament_selection(pop, fitnesses, TOURNAMENT_K)
            p2 = tournament_selection(pop, fitnesses, TOURNAMENT_K)
            child = ordered_crossover(p1, p2, start_idx)
            mutate_swap(child, MUTATION_RATE)
            new_pop.append(child)

        pop = new_pop

    return best_route, best_dist

# Simple TSP Solver

In [ ]:
def solve_tsp_simple(uploaded_file=None, start_city_index=0):

    # Load data
    cities, dist_matrix = read_distance_matrix(uploaded_file)
    print(f"📍 Loaded {len(cities)} cities")

    # Run genetic algorithm
    print("🔧 Running Genetic Algorithm...")
    start_time = time.time()
    route, distance = genetic_tsp(dist_matrix, start_city_index)
    end_time = time.time()

    # Display results
    print("\n" + "="*50)
    print("✅ TSP SOLUTION FOUND")
    print("="*50)
    print(f"⏱️  Computation time: {end_time - start_time:.2f} seconds")
    print(f"📏 Optimal distance: {distance:.3f}")
    print(f"🏁 Starting city: {cities[start_city_index]} (Index {start_city_index})")
    print("\n🛣️  OPTIMAL ROUTE:")
    for step, city_idx in enumerate(route):
        print(f"   {step + 1:2d}. {cities[city_idx]} (City {city_idx})")

    return route, distance, cities

# Main

In [ ]:
def main_simple():

    cities, dist_matrix = read_distance_matrix(uploaded)

    # Select start city
    start_idx = 0
    print(f"🏁 Starting from: {cities[start_idx]} (Index {start_idx})")

    # Run genetic algorithm
    start_time = time.time()
    route, distance = genetic_tsp(dist_matrix, start_idx)
    end_time = time.time()

    # Display results
    print("✅ RESULTS")
    print(f"⏱️  Time: {end_time - start_time:.2f} seconds")
    print(f"📏 Total Distance: {distance:.2f}")

    print(f"\n🛣️  OPTIMAL ROUTE:")
    route_display = [f"{cities[i]}" for i in route]
    print(" → ".join(route_display) + f" → {cities[route[0]]}")

    return route, distance, cities

# Run the simple main function
if __name__ == "__main__":
    route, distance, cities = main_simple()

DataFrame shape: (27, 28)

First few rows of the dataframe:
        Unnamed: 0  Albuquerque, NM  Atlanta, GA  Austin, TX  Baltimore, MD  \
0  Albuquerque, NM                0         1494        1330           1295   
1      Atlanta, GA             1494            0        2947           1175   
2       Austin, TX             1330         2947           0           2155   
3    Baltimore, MD             1295         1175        2155              0   
4       Boston, MA             1838         2006        1785            579   

   Boston, MA  Charlotte, NC  Chicago, IL  Cleveland, OH  Columbus, OH  ...  \
0        1838           2369          666           1438           530  ...   
1        2006            389         2934            762          2099  ...   
2        1785           1221         2813           1329          1700  ...   
3         579            692         1380           2262           264  ...   
4           0           1951         1004           2346          2931

Générer la solution de référence avec OR-Tools

In [ ]:
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.7/27.7 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 23.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.6.15 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 6.31.1 which is incompatible.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.

In [ ]:
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
import numpy as np

def tsp_solution_reference_ortools(distance_matrix):
    n = len(distance_matrix)

    # Manager pour indexer les noeuds
    manager = pywrapcp.RoutingIndexManager(n, 1, 0)  # 1 véhicule, départ=0
    routing = pywrapcp.RoutingModel(manager)

    # Callback de distance (float -> int via arrondi si nécessaire)
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return int(round(distance_matrix[from_node][to_node]))

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Paramètres de recherche
    search_parameters = pywrapcp.DefaultRoutin_

Etape 1 du projet : Tableau des résultats expérimentaux

In [ ]:
import pandas as pd
import numpy as np
import time
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

# -------------------------
# Fonction OR-Tools corrigée
# -------------------------
def tsp_solution_reference_ortools(distance_matrix):
    """
    Calcule la distance minimale exacte du TSP avec Google OR-Tools.
    Renvoie la distance optimale et le chemin complet.
    """
    n = len(distance_matrix)
    manager = pywrapcp.RoutingIndexManager(n, 1, 0)  # 1 véhicule, départ=0
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return int(round(distance_matrix[from_node][to_node]))

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    search_parameters.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    search_parameters.time_limit.seconds = 10  # Limite de 10 secondes

    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        # Extraire le chemin optimal
        route = []
        index = routing.Start(0)
        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            route.append(node)
            index = solution.Value(routing.NextVar(index))
        route.append(route[0])  # Retour à la ville de départ

        # Calculer la distance exacte
        total_distance = sum(distance_matrix[route[i]][route[i+1]] for i in range(len(route)-1))
        return total_distance
    else:
        return None

# -------------------------
# Configuration des instances
# -------------------------
instances = [
    {"designation": "TSP-1", "n_objects": 10},
    {"designation": "TSP-2", "n_objects": 15},
    {"designation": "TSP-3", "n_objects": 27}
]

results = []

# Fixer le seed pour la reproductibilité
np.random.seed(42)

# -------------------------
# Boucle d'expérimentation
# -------------------------
for inst in instances:
    name = inst["designation"]
    n = inst["n_objects"]

    print(f"\n🔹 Exécution de l'instance {name} avec {n} villes...")

    # 1️⃣ Créer matrice de distances symétrique
    dist = np.random.rand(n, n) * 100
    dist = (dist + dist.T) / 2
    np.fill_diagonal(dist, 0)

    # 2️⃣ Solution de référence OR-Tools
    start_ref = time.time()
    ref_sol = tsp_solution_reference_ortools(dist)
    ref_time = time.time() - start_ref

    # 3️⃣ Solution métaheuristique (GA)
    start_meta = time.time()
    _, meta_sol = genetic_tsp(dist, start_idx=0)
    meta_time = time.time() - start_meta

    # 4️⃣ Calcul du GAP (%)
    gap = abs(meta_sol - ref_sol) / ref_sol * 100 if ref_sol and ref_sol > 0 else None

    # 5️⃣ Stocker les résultats
    results.append({
        "Désignation": name,
        "Nombre de villes": n,
        "Solution de référence": round(ref_sol, 2) if ref_sol else "N/A",
        "Solution métaheuristique": round(meta_sol, 2),
        "GAP (%)": round(gap, 2) if gap is not None else "N/A",
        "CPU Time GA (s)": round(meta_time, 2),
        "CPU Time OR-Tools (s)": round(ref_time, 2)
    })

# -------------------------
# Affichage et sauvegarde
# -------------------------
df_results = pd.DataFrame(results)
display(df_results)

output_file = "comparaison_TSP_GAP.xlsx"
df_results.to_excel(output_file, index=False)
print(f"\n✅ Tableau sauvegardé : {output_file}")



🔹 Exécution de l'instance TSP-1 avec 10 villes...

🔹 Exécution de l'instance TSP-2 avec 15 villes...

🔹 Exécution de l'instance TSP-3 avec 27 villes...


,Désignation,Nombre de villes,Solution de référence,Solution métaheuristique,GAP (%),CPU Time GA (s),CPU Time OR-Tools (s)
0,TSP-1,10,238.92,238.92,0.00,2.08,10.02
1,TSP-2,15,331.83,347.10,4.60,2.16,10.00
2,TSP-3,27,398.49,540.54,35.65,3.20,10.00



✅ Tableau sauvegardé : comparaison_TSP_GAP.xlsx


méthode hybride : GA + Tabou

In [ ]:
# -*- coding: utf-8 -*-
"""TSP GA + Tabou

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1AQbTYNwqUBRIw4IOGlW1_rEOMyWRYFVk
"""

# Installation des dépendances nécessaires
!pip install openpyxl ortools

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import threading
import time
import os
from IPython.display import display, clear_output
import ipywidgets as widgets
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

# -----------------------
# Configurations GA + Tabou
# -----------------------
POP_SIZE = 200
GENERATIONS = 600
TOURNAMENT_K = 8
MUTATION_RATE = 0.1
ELITISM = 30
TABU_TENURE = 15
TABU_ITERATIONS = 100
RANDOM_SEED = 42

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# -----------------------
# Dataset helper
# -----------------------
def create_sample_data():
    """Crée des données d'exemple si aucun fichier n'est uploadé"""
    cities = [f"Ville_{i}" for i in range(10)]
    n = len(cities)
    dist = np.random.rand(n, n) * 100
    dist = (dist + dist.T) / 2
    np.fill_diagonal(dist, 0)
    return cities, dist

def read_distance_matrix(uploaded_file=None):
    """Lit la matrice de distance depuis un fichier uploadé ou utilise des données d'exemple"""
    if uploaded_file is None:
        print("Utilisation de données d'exemple...")
        return create_sample_data()

    try:
        if uploaded_file.name.endswith('.xlsx'):
            df = pd.read_excel(uploaded_file, engine="openpyxl")
        elif uploaded_file.name.endswith('.csv'):
            df = pd.read_csv(uploaded_file)
        else:
            raise ValueError("Format de fichier non supporté")

        # Essayer de lire comme matrice carrée
        try:
            df2 = df.set_index(df.columns[0])
            if df2.shape[0] == df2.shape[1]:
                cities = list(df2.index.astype(str))
                dist = df2.values.astype(float)
                print(f"Matrice carrée détectée: {len(cities)} villes")
                return cities, dist
        except:
            pass

        # Essayer de lire comme liste d'arêtes
        lowered = [c.lower() for c in df.columns.astype(str)]
        if "city1" in lowered and "city2" in lowered and ("distance" in lowered or "dist" in lowered):
            cols = df.columns.astype(str)
            c1 = cols[[i for i, v in enumerate(lowered) if v=="city1"][0]]
            c2 = cols[[i for i, v in enumerate(lowered) if v=="city2"][0]]
            cd = cols[[i for i, v in enumerate(lowered) if v in ["distance","dist"]][0]]
            edges = df[[c1,c2,cd]].copy()
            cities = sorted(list(set(edges[c1].astype(str)).union(set(edges[c2].astype(str)))))
            n = len(cities)
            idx = {c:i for i,c in enumerate(cities)}
            dist = np.full((n,n), np.inf)
            for r in range(n): dist[r,r]=0.0
            for _, row in edges.iterrows():
                i,j,d = idx[str(row[c1])], idx[str(row[c2])], float(row[cd])
                dist[i,j] = dist[j,i] = d
            print(f"Liste d'arêtes détectée: {len(cities)} villes")
            return cities, dist

        raise ValueError("Format du fichier non reconnu.")

    except Exception as e:
        print(f"Erreur lecture fichier: {e}, utilisation de données d'exemple")
        return create_sample_data()

# -----------------------
# Fonctions de base pour TSP
# -----------------------
def route_distance(route, dist_matrix):
    """Calcule la distance totale d'un parcours"""
    d = sum(dist_matrix[route[i], route[i+1]] for i in range(len(route)-1))
    d += dist_matrix[route[-1], route[0]]
    return float(d)

# -----------------------
# Fonctions GA (inchangées)
# -----------------------
def initial_population(n_cities, pop_size, start_idx):
    population = []
    nodes = [i for i in range(n_cities) if i != start_idx]
    for _ in range(pop_size):
        perm = nodes.copy()
        random.shuffle(perm)
        population.append([start_idx]+perm)
    return population

def tournament_selection(pop, fitnesses, k):
    selected = random.sample(range(len(pop)), k)
    best = min(selected, key=lambda i: fitnesses[i])
    return pop[best].copy()

def ordered_crossover(p1, p2, start_idx):
    n = len(p1)
    child = [-1]*n
    child[0] = start_idx
    a, b = random.randint(1,n-2), random.randint(1,n-1)
    if b<=a: a,b=b,a+1
    child[a:b] = p1[a:b]
    p2_iter = [x for x in p2[1:] if x not in child]
    pos=1
    for v in p2_iter:
        while pos<n and child[pos]!=-1: pos+=1
        if pos<n: child[pos]=v; pos+=1
    for pos in range(1,n):
        if child[pos]==-1:
            for v in p2[1:]:
                if v not in child:
                    child[pos]=v
                    break
    return child

def mutate_swap(route, mutation_rate):
    n = len(route)
    for i in range(1,n):
        if random.random()<mutation_rate:
            j = random.randint(1,n-1)
            route[i], route[j] = route[j], route[i]

# -----------------------
# Fonctions Tabou Search
# -----------------------
def two_opt_move(route, i, j):
    """Effectue un mouvement 2-opt"""
    new_route = route.copy()
    new_route[i:j+1] = reversed(route[i:j+1])
    return new_route

def tabu_search(initial_route, dist_matrix, max_iterations=TABU_ITERATIONS, tabu_tenure=TABU_TENURE):
    """
    Recherche Tabou pour améliorer une solution

    Args:
        initial_route: Solution initiale
        dist_matrix: Matrice des distances
        max_iterations: Nombre maximum d'itérations
        tabu_tenure: Durée de la liste tabou

    Returns:
        Meilleure solution trouvée
    """
    current_solution = initial_route.copy()
    best_solution = current_solution.copy()
    best_distance = route_distance(best_solution, dist_matrix)

    # Liste tabou: stocke les mouvements interdits
    tabu_list = []

    for iteration in range(max_iterations):
        best_neighbor = None
        best_neighbor_distance = float('inf')
        best_move = None

        # Génère les voisins par mouvements 2-opt
        for i in range(1, len(current_solution) - 1):
            for j in range(i + 1, len(current_solution)):
                if j - i == 1:
                    continue

                neighbor = two_opt_move(current_solution, i, j)
                neighbor_distance = route_distance(neighbor, dist_matrix)
                move = (min(i, j), max(i, j))

                # Critère d'aspiration
                aspiration_criteria = neighbor_distance < best_distance

                if (move not in tabu_list) or aspiration_criteria:
                    if neighbor_distance < best_neighbor_distance:
                        best_neighbor = neighbor
                        best_neighbor_distance = neighbor_distance
                        best_move = move

        if best_neighbor is None:
            break

        # Met à jour la solution courante
        current_solution = best_neighbor
        current_distance = best_neighbor_distance

        # Met à jour la meilleure solution
        if current_distance < best_distance:
            best_solution = current_solution.copy()
            best_distance = current_distance

        # Met à jour la liste tabou
        tabu_list.append(best_move)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)

    return best_solution, best_distance

# -----------------------
# Algorithme Hybride GA + Tabou CORRIGÉ
# -----------------------
def hybrid_ga_tabu(dist_matrix, start_idx, pop_size=POP_SIZE, generations=GENERATIONS):
    """
    Algorithme hybride combinant GA et Tabou Search - VERSION CORRIGÉE
    """
    n = dist_matrix.shape[0]

    # Réinitialiser le seed pour éviter les biais
    random.seed(time.time())
    np.random.seed(int(time.time() * 1000) % 2**32)

    # Phase 1: Algorithme Génétique
    pop = initial_population(n, pop_size, start_idx)
    best_route, best_dist = None, float('inf')

    for gen in range(generations):
        fitnesses = [route_distance(r, dist_matrix) for r in pop]
        min_idx = int(np.argmin(fitnesses))
        if fitnesses[min_idx] < best_dist:
            best_dist = fitnesses[min_idx]
            best_route = pop[min_idx].copy()

        new_pop = []
        sorted_idx = np.argsort(fitnesses)

        # Élitisme
        for e in range(min(ELITISM, len(sorted_idx))):
            new_pop.append(pop[int(sorted_idx[e])].copy())

        # Reproduction
        while len(new_pop) < pop_size:
            p1 = tournament_selection(pop, fitnesses, TOURNAMENT_K)
            p2 = tournament_selection(pop, fitnesses, TOURNAMENT_K)
            child = ordered_crossover(p1, p2, start_idx)
            mutate_swap(child, MUTATION_RATE)
            new_pop.append(child)

        pop = new_pop

        # Application périodique de Tabou Search SUR LES MEILLEURES SOLUTIONS SEULEMENT
        if gen % 25 == 0 and gen > 50:  # Commencer après 50 générations
            elite_size = max(2, pop_size // 20)  # Moins d'individus pour Tabou
            for i in range(elite_size):
                # Utiliser un nombre d'itérations variable
                tabu_iter = min(30 + gen // 10, 100)
                improved_route, improved_dist = tabu_search(
                    pop[i], dist_matrix,
                    max_iterations=tabu_iter,
                    tabu_tenure=max(5, TABU_TENURE // 2)
                )
                if improved_dist < route_distance(pop[i], dist_matrix):
                    pop[i] = improved_route

    # Phase 2: Recherche Tabou finale PLUS AGRESSIVE
    final_route, final_dist = tabu_search(
        best_route, dist_matrix,
        max_iterations=300,  # Plus d'itérations
        tabu_tenure=TABU_TENURE
    )

    # Validation de la solution
    if not is_valid_route(final_route, n):
        print("⚠️  Route invalide détectée, utilisation de la solution GA")
        return best_route, best_dist

    if final_dist < best_dist:
        best_route = final_route
        best_dist = final_dist

    return best_route, best_dist

def is_valid_route(route, n_cities):
    """Vérifie si une route est valide (toutes les villes visitées une fois)"""
    if len(route) != n_cities:
        return False
    if len(set(route)) != n_cities:
        return False
    if route[0] != 0:  # Doit commencer par la ville de départ
        return False
    return True

# -----------------------
# Fonction Tabou Search CORRIGÉE
# -----------------------
def tabu_search(initial_route, dist_matrix, max_iterations=TABU_ITERATIONS, tabu_tenure=TABU_TENURE):
    """
    Recherche Tabou - VERSION CORRIGÉE avec diversité
    """
    current_solution = initial_route.copy()
    best_solution = current_solution.copy()
    best_distance = route_distance(best_solution, dist_matrix)

    tabu_list = []
    no_improvement_count = 0

    for iteration in range(max_iterations):
        best_neighbor = None
        best_neighbor_distance = float('inf')
        best_move = None

        # Génère les voisins avec plus de diversité
        neighbors_tested = 0
        max_neighbors = min(100, len(current_solution) * 3)  # Limite le nombre de voisins testés

        while neighbors_tested < max_neighbors:
            i = random.randint(1, len(current_solution) - 2)
            j = random.randint(i + 1, len(current_solution) - 1)

            neighbor = two_opt_move(current_solution, i, j)
            neighbor_distance = route_distance(neighbor, dist_matrix)
            move = (min(i, j), max(i, j))

            aspiration_criteria = neighbor_distance < best_distance * 0.98  # Critère plus strict

            if (move not in tabu_list) or aspiration_criteria:
                if neighbor_distance < best_neighbor_distance:
                    best_neighbor = neighbor
                    best_neighbor_distance = neighbor_distance
                    best_move = move

            neighbors_tested += 1

        if best_neighbor is None:
            # Pas de voisin améliorant trouvé, essayer un mouvement aléatoire
            i = random.randint(1, len(current_solution) - 2)
            j = random.randint(i + 1, len(current_solution) - 1)
            best_neighbor = two_opt_move(current_solution, i, j)
            best_neighbor_distance = route_distance(best_neighbor, dist_matrix)
            best_move = (i, j)

        # Met à jour la solution courante
        current_solution = best_neighbor
        current_distance = best_neighbor_distance

        # Met à jour la meilleure solution
        if current_distance < best_distance:
            best_solution = current_solution.copy()
            best_distance = current_distance
            no_improvement_count = 0
        else:
            no_improvement_count += 1

        # Critère d'arrêt précoce
        if no_improvement_count > max_iterations // 3:
            break

        # Met à jour la liste tabou
        tabu_list.append(best_move)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)

    return best_solution, best_distance

# -----------------------
# Nouvelle expérimentation avec vérifications
# -----------------------
def run_comparison_experiment_corrected():
    """Exécute l'expérimentation comparative - VERSION CORRIGÉE"""

    instances = [
        {"designation": "TSP-1", "n_objects": 10},
        {"designation": "TSP-2", "n_objects": 15},
        {"designation": "TSP-3", "n_objects": 27},
        {"designation": "TSP-4", "n_objects": 20},  # Instance supplémentaire
    ]

    results = []

    for inst in instances:
        name = inst["designation"]
        n = inst["n_objects"]

        print(f"\n🔹 Exécution de l'instance {name} avec {n} villes...")

        # Créer matrice de distances avec seed différent pour chaque instance
        seed = hash(name) % 10000
        np.random.seed(seed)
        dist = np.random.rand(n, n) * 100
        dist = (dist + dist.T) / 2
        np.fill_diagonal(dist, 0)

        # 1️⃣ Solution de référence OR-Tools
        print("  - Calcul solution OR-Tools...")
        start_ref = time.time()
        ref_sol = tsp_solution_reference_ortools(dist)
        ref_time = time.time() - start_ref

        # Réinitialiser les seeds pour les méthodes métaheuristiques
        random.seed(time.time())
        np.random.seed(int(time.time() * 1000) % 2**32)

        # 2️⃣ Solution GA seul
        print("  - Calcul solution GA...")
        start_ga = time.time()
        ga_route, ga_sol = genetic_tsp(dist, start_idx=0)
        ga_time = time.time() - start_ga

        # Réinitialiser à nouveau
        random.seed(time.time() + 1)
        np.random.seed(int(time.time() * 1000 + 1) % 2**32)

        # 3️⃣ Solution Tabou seul
        print("  - Calcul solution Tabou...")
        start_tabu = time.time()
        initial_route = list(range(n))
        random.shuffle(initial_route)
        tabu_route, tabu_sol = tabu_search(initial_route, dist, max_iterations=500)
        tabu_time = time.time() - start_tabu

        # Réinitialiser à nouveau
        random.seed(time.time() + 2)
        np.random.seed(int(time.time() * 1000 + 2) % 2**32)

        # 4️⃣ Solution Hybride GA + Tabou
        print("  - Calcul solution hybride GA+Tabou...")
        start_hybrid = time.time()
        hybrid_route, hybrid_sol = hybrid_ga_tabu(dist, start_idx=0)
        hybrid_time = time.time() - start_hybrid

        # VÉRIFICATION des solutions
        print(f"  - Vérification des solutions:")
        print(f"    OR-Tools: {ref_sol:.2f}" if ref_sol else "    OR-Tools: N/A")
        print(f"    GA: {ga_sol:.2f}")
        print(f"    Tabou: {tabu_sol:.2f}")
        print(f"    Hybride: {hybrid_sol:.2f}")

        # 5️⃣ Calcul des GAPs
        if ref_sol and ref_sol > 0:
            gap_ga = abs(ga_sol - ref_sol) / ref_sol * 100
            gap_tabu = abs(tabu_sol - ref_sol) / ref_sol * 100
            gap_hybrid = abs(hybrid_sol - ref_sol) / ref_sol * 100
        else:
            gap_ga = gap_tabu = gap_hybrid = "N/A"

        # Stocker les résultats
        results.append({
            "Désignation": name,
            "Nombre de villes": n,
            "Solution OR-Tools": round(ref_sol, 2) if ref_sol else "N/A",
            "Solution GA": round(ga_sol, 2),
            "Solution Tabou": round(tabu_sol, 2),
            "Solution Hybride": round(hybrid_sol, 2),
            "GAP GA (%)": round(gap_ga, 2) if gap_ga != "N/A" else "N/A",
            "GAP Tabou (%)": round(gap_tabu, 2) if gap_tabu != "N/A" else "N/A",
            "GAP Hybride (%)": round(gap_hybrid, 2) if gap_hybrid != "N/A" else "N/A",
            "CPU Time GA (s)": round(ga_time, 2),
            "CPU Time Tabou (s)": round(tabu_time, 2),
            "CPU Time Hybride (s)": round(hybrid_time, 2),
            "CPU Time OR-Tools (s)": round(ref_time, 2)
        })

    return pd.DataFrame(results)

# -----------------------
# Exécution de la version corrigée
# -----------------------
print("🔬 Début de l'expérimentation comparative CORRIGÉE...")
df_results_corrected = run_comparison_experiment_corrected()

print("\n" + "="*80)
print("📊 RÉSULTATS CORRIGÉS - COMPARAISON RÉELLE")
print("="*80)
display(df_results_corrected)

# Analyse des résultats
print("\n🎯 ANALYSE DES RÉSULTATS CORRIGÉS :")
print("• Les GAPs devraient maintenant montrer des différences réelles entre les méthodes")
print("• La solution hybride devrait être meilleure que GA seul mais pas forcément égale à OR-Tools")
print("• Les temps d'exécution reflètent la complexité réelle de chaque méthode")

🔬 Début de l'expérimentation comparative CORRIGÉE...

🔹 Exécution de l'instance TSP-1 avec 10 villes...
  - Calcul solution OR-Tools...
  - Calcul solution GA...
  - Calcul solution Tabou...
  - Calcul solution hybride GA+Tabou...
  - Vérification des solutions:
    OR-Tools: 277.03
    GA: 277.94
    Tabou: 276.14
    Hybride: 276.14

🔹 Exécution de l'instance TSP-2 avec 15 villes...
  - Calcul solution OR-Tools...
  - Calcul solution GA...
  - Calcul solution Tabou...
  - Calcul solution hybride GA+Tabou...
  - Vérification des solutions:
    OR-Tools: 346.62
    GA: 373.16
    Tabou: 346.62
    Hybride: 346.62

🔹 Exécution de l'instance TSP-3 avec 27 villes...
  - Calcul solution OR-Tools...
  - Calcul solution GA...
  - Calcul solution Tabou...
  - Calcul solution hybride GA+Tabou...
  - Vérification des solutions:
    OR-Tools: 516.04
    GA: 661.07
    Tabou: 515.53
    Hybride: 515.53

🔹 Exécution de l'instance TSP-4 avec 20 villes...
  - Calcul solution OR-Tools...
  - Calcul s

,Désignation,Nombre de villes,Solution OR-Tools,Solution GA,Solution Tabou,Solution Hybride,GAP GA (%),GAP Tabou (%),GAP Hybride (%),CPU Time GA (s),CPU Time Tabou (s),CPU Time Hybride (s),CPU Time OR-Tools (s)
0,TSP-1,10,277.03,277.94,276.14,276.14,0.33,0.32,0.32,2.05,0.05,3.27,10.0
1,TSP-2,15,346.62,373.16,346.62,346.62,7.65,0.00,0.00,2.33,0.10,5.55,10.0
2,TSP-3,27,516.04,661.07,515.53,515.53,28.10,0.10,0.10,4.56,0.26,8.42,10.0
3,TSP-4,20,361.34,380.23,361.34,361.34,5.23,0.00,0.00,3.06,0.25,6.51,10.0



🎯 ANALYSE DES RÉSULTATS CORRIGÉS :
• Les GAPs devraient maintenant montrer des différences réelles entre les méthodes
• La solution hybride devrait être meilleure que GA seul mais pas forcément égale à OR-Tools
• Les temps d'exécution reflètent la complexité réelle de chaque méthode
